Коллаб с удобным выводом чанков https://youtu.be/Tk2W92d30jY

# Code

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13


In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


HISTORY = []



In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
#SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO
SELECT_MODEL_GPT =MODEL_GPT_3_5_TURBO_16K

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

def load_search_indexes(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    #text = response.content.decode('unicode_escape')
    text = response.text
    headers_to_split_on = [
        ("#", "Header1"),
        ("##", "Header2"),
        ("###", "Header3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    #записываем получившийся список из чанков в файл
    with open('KIA_ByChuncks.txt', 'w') as f:
      f.writelines(f"{item}\n" for item in md_header_splits)
    f.close()

    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())

    return vectordateBase


In [ ]:
#Это ссылка на мой файл БЗ от 20.09
knowledge_base = load_search_indexes('https://docs.google.com/document/d/1X9ojw6qZwlo_J4kYllFHnUWS76WSgrr_75SJjAhVI0Y/edit?usp=sharing')

RateLimitError: ignored

## основная функция

In [ ]:

def _summarize_topic( dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content



def answer_index(topic, temp=0.1, top_similar_documents=5):  #я поставила 3 документа, было 5
        summarize_text = ''
        #я Убираю саммари для тестирования
        #яif len(HISTORY) > 0:
        #я    summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
        #я        [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
        #я    print(f'САММАРИ \n=== {summarize_text} \n')

        # Добавляем явное разделение между историей диалога и текущим вопросом
        #яinput_text = summarize_text + "\n\nТекущий вопрос: " + topic
        input_text = topic

        docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)
        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f'\n==================Отрывок документа №{i + 1}=====================\n{content}\n'
                if verbose == 1:
                  print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                  print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                  print(f'\n{content}\n')
                responses.append(response)


        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа пользователю: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        #яHISTORY.append((topic, answer if answer is not None else ''))

        return insert_newlines(answer)  # возвращает ответ


In [ ]:
def run_dialog(system_doc_url, knowledge_base_url):

    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer = answer_index(user_question)
        print('\nМенеджер: ', answer+'\n\n')

    return

In [ ]:
# Промпт моделей машин
prompt = load_document_text ('https://docs.google.com/document/d/1i8HA7cX4Ut-tb9rf8wOgERU7lLe66xJYscizGtSSJl0')

In [ ]:
temperature=0.3
verbose=1

#HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog(prompt, knowledge_base)


Клиент: Чем отличаются комплектации Comfort Kia Ceed

=====================Отрывок документа №1=====================

=== score = 0.2215176820755005  Metadata документа ------------ {'Header 1': 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/', 'Header 2': 'Комфорт'}

Комфорт
Достоин быть вашим. В Kia Ceed собрано все самое необходимое, чтобы  ваше вождение было максимально удобным и комфортным. Настройте все под себя и разместитесь удобнее. Превратите автомобиль в место, в котором хочется находиться как можно дольше.
Климат-контроль. Обогрев передних и задних сидений. Память настроек (IMS). Подрулевые рычаги переключения передач. Система бесключевого доступа Smart Key и запуск двигателя кнопкой. Климат-контроль. Для поддержания нужного климата в салоне можно выбрать как обычный кондиционер с ручным управлением, так и двухзонный климат-контроль для настройки индивидуального климата для водителя и пассажира.. Обогрев передних и задних сидений. Обогрев передних и задних сидений п

In [ ]:
HISTORY = []

## Собственно диалог

In [ ]:
temperature=0.1
verbose=1

#HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog(prompt, knowledge_base)


Клиент: Какая минимальная комплектация Carnival и ее цена

=====================Отрывок документа №1=====================

=== score = 0.32416218519210815  Metadata документа ------------ {'Header 1': 'Kia Carnival 2022 - https://www.kia.ru/models/carnival/desc/', 'Header 2': 'Комплектации', 'Header 3': 'Комплектация: Kia Carnival Premium+ 3.5 MPI / 249 л.с. / Бензин / Автомат, 8AT / Передний привод'}

Комплектация: Kia Carnival Premium+ 3.5 MPI / 249 л.с. / Бензин / Автомат, 8AT / Передний привод
Цена: 5 324 900 ₽
Стандартное оборудование : Светодиодные дневные ходовые огни, Дополнительный светодиодный стоп-сигнал, Два разъёма USB для зарядки мобильных устройств в передней панели, Разъём USB для зарядки мобильных устройств в боковой части спинки передних сидений, Дополнительная розетка 12V в багажном отделении, Крепления для детского кресла ISOFIX на втором и третьем ряду сидений, Система помощи при старте на подъёме (HAC), Система активного управления стабилизацией (VSM), Фронтальны

# Мои дополнения для автоматизации тестирования

# Просто эксперименты

In [ ]:
topic = 'Какая цена у Сид Комфорт'
docs = knowledge_base.similarity_search_with_score(topic, k=5)
responses = []
for i, (doc, score) in enumerate(docs):
   if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
      print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')


=== score = 0.34520357847213745  Metadata документа ------------ {'Header1': 'Kia Carnival 2022 - https://www.kia.ru/models/carnival/desc/', 'Header2': 'Интерьер'}
=== score = 0.3497665822505951  Metadata документа ------------ {'Header1': 'Kia Cerato 2022 - https://www.kia.ru/models/cerato/desc/', 'Header2': 'Комфорт'}
=== score = 0.3562178611755371  Metadata документа ------------ {'Header1': 'Kia Soul 2022 - https://www.kia.ru/models/soul/desc/', 'Header2': 'Комфорт'}
=== score = 0.36482396721839905  Metadata документа ------------ {'Header1': 'Kia K9 2022 - https://www.kia.ru/models/k9/desc/', 'Header2': 'Комфорт'}
=== score = 0.3661295771598816  Metadata документа ------------ {'Header1': 'Kia Carnival 2022 - https://www.kia.ru/models/carnival/desc/', 'Header2': 'Комфорт'}


In [ ]:
topic = 'Какая цена у Сид Комфорт'
docs = knowledge_base.similarity_search_with_score(topic, k= 5, filter=dict(Header2 = 'Комфорт'))
responses = []
for i, (doc, score) in enumerate(docs):
   if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
      print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')


=== score = 0.349797785282135  Metadata документа ------------ {'Header1': 'Kia Cerato 2022 - https://www.kia.ru/models/cerato/desc/', 'Header2': 'Комфорт'}
=== score = 0.3563109338283539  Metadata документа ------------ {'Header1': 'Kia Soul 2022 - https://www.kia.ru/models/soul/desc/', 'Header2': 'Комфорт'}
=== score = 0.3649287223815918  Metadata документа ------------ {'Header1': 'Kia K9 2022 - https://www.kia.ru/models/k9/desc/', 'Header2': 'Комфорт'}
=== score = 0.3662051558494568  Metadata документа ------------ {'Header1': 'Kia Carnival 2022 - https://www.kia.ru/models/carnival/desc/', 'Header2': 'Комфорт'}
=== score = 0.36660271883010864  Metadata документа ------------ {'Header1': 'Kia Rio 2022 - https://www.kia.ru/models/rio/desc/', 'Header2': 'Комфорт'}


In [ ]:
topic = 'Какая цена у Ceed Comfort'
docs = knowledge_base.similarity_search_with_score(topic, k= 5, filter=dict(Header1 = 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/'))
responses = []
for i, (doc, score) in enumerate(docs):
   if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
      print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')

=== score = 0.2673403322696686  Metadata документа ------------ {'Header1': 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/', 'Header2': 'Комплектации', 'Header3': 'Комплектация: Kia Ceed Comfort 1.6 MPI / 128 л.с. / Бензин / Автомат, 6AT / Передний привод'}
=== score = 0.282987117767334  Metadata документа ------------ {'Header1': 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/', 'Header2': 'Комфорт'}
=== score = 0.3074369430541992  Metadata документа ------------ {'Header1': 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/', 'Header2': 'Комплектации', 'Header3': 'Комплектация: Kia Ceed Premium 1.6 MPI / 128 л.с. / Бензин / Автомат, 6AT / Передний привод'}
=== score = 0.30959129333496094  Metadata документа ------------ {'Header1': 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/', 'Header2': 'Комплектации', 'Header3': 'Комплектация: Kia Ceed Luxe 1.6 MPI / 128 л.с. / Бензин / Автомат, 6AT / Передний привод'}
=== score = 0.3104107081890106  Metadata документа ---

In [ ]:
topic = 'Какая цена у Сид Комфорт'
docs = knowledge_base.similarity_search_with_score(topic, k= 5, filter=dict(Header1 = 'Kia Ceed 2022 - https://www.kia.ru/models/ceed/desc/', Header2= 'Комплектации'))
responses = []
for i, (doc, score) in enumerate(docs):
   if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
      print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')


In [ ]:
from langchain.schema import Document
embeddings = OpenAIEmbeddings()

list_of_documents = [
    Document(page_content="foo", metadata=dict(page=1, header1 = 'dfdcfvf')),
    Document(page_content="bar", metadata=dict(page=1, header1 = 'dfdcfvf')),
    Document(page_content="foo", metadata=dict(page=2, header1 = 'wwwwww')),
    Document(page_content="barbar", metadata=dict(page=2, header1 = 'dfdcfvf')),
    Document(page_content="foo", metadata=dict(page=3, header1 = 'wwwwww')),
    Document(page_content="bar burr", metadata=dict(page=3, header1 = 'dfdcfvf')),
    Document(page_content="foo", metadata=dict(page=4, header1 = 'wwwwww')),
    Document(page_content="bar bruh", metadata=dict(page=4, header1 = 'dfdcfvf')),
]
db = FAISS.from_documents(list_of_documents, embeddings)
results_with_scores = db.similarity_search_with_score("foo")
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1, 'header1': 'dfdcfvf'}, Score: 7.443106359386348e-15
Content: foo, Metadata: {'page': 2, 'header1': 'wwwwww'}, Score: 7.443106359386348e-15
Content: foo, Metadata: {'page': 3, 'header1': 'wwwwww'}, Score: 7.443106359386348e-15
Content: foo, Metadata: {'page': 4, 'header1': 'wwwwww'}, Score: 7.443106359386348e-15


In [ ]:
list_of_documents

[Document(page_content='foo', metadata={'page': 1, 'header1': 'dfdcfvf'}),
 Document(page_content='bar', metadata={'page': 1, 'header1': 'dfdcfvf'}),
 Document(page_content='foo', metadata={'page': 2, 'header1': 'wwwwww'}),
 Document(page_content='barbar', metadata={'page': 2, 'header1': 'dfdcfvf'}),
 Document(page_content='foo', metadata={'page': 3, 'header1': 'wwwwww'}),
 Document(page_content='bar burr', metadata={'page': 3, 'header1': 'dfdcfvf'}),
 Document(page_content='foo', metadata={'page': 4, 'header1': 'wwwwww'}),
 Document(page_content='bar bruh', metadata={'page': 4, 'header1': 'dfdcfvf'})]

In [ ]:
results_with_scores = db.similarity_search_with_score("foo",)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1, 'header1': 'dfdcfvf'}, Score: 1.4206958439899608e-05
Content: foo, Metadata: {'page': 2}, Score: 1.4206958439899608e-05
Content: foo, Metadata: {'page': 3}, Score: 1.4206958439899608e-05
Content: foo, Metadata: {'page': 4}, Score: 1.4206958439899608e-05


In [ ]:
results_with_scores = db.similarity_search_with_score("foo", filter=dict(page=1))
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1, 'header1': 'dfdcfvf'}, Score: 7.443106359386348e-15
Content: bar, Metadata: {'page': 1}, Score: 0.3131275773048401


In [ ]:
results_with_scores = db.similarity_search_with_score("foo", filter=dict(header1='wwwwww'))
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 2, 'header1': 'wwwwww'}, Score: 7.443106359386348e-15
Content: foo, Metadata: {'page': 3, 'header1': 'wwwwww'}, Score: 7.443106359386348e-15
Content: foo, Metadata: {'page': 4, 'header1': 'wwwwww'}, Score: 7.443106359386348e-15


# Автоматическое заполнение файла ответами

In [ ]:
#authenticate, and create the interface to Sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1dlzmaiKBX8ZxIc-OM9XUrv2vkQVgY1Gs5SYPm9ZZQ8s/edit?usp=sharing')

In [ ]:
#получаем список всех листов файла
worksheet_list = worksheet.worksheets()
print(worksheet_list)

[<Worksheet 'Инструкция !!!' id:839332776>, <Worksheet 'Вопросы' id:1214543959>, <Worksheet 'Сводная таблица 1' id:123014530>, <Worksheet 'Статистика' id:620905567>, <Worksheet 'Вопросы v1' id:2129939177>, <Worksheet 'Типовые проблемы' id:816259107>, <Worksheet 'Доп вопросы' id:1850819785>, <Worksheet 'Чанки' id:238143192>, <Worksheet 'from bot' id:302818179>]


In [ ]:
#обращаемся к нашему листу
sheet = worksheet.worksheet("Вопросы")
#получаем все строки листа
rows = sheet.get_all_values()

In [ ]:
#задаем номер ВОПРОСА (то есть значения, указанного в столбце A), которые проверяем
start_index = 186
end_index= 263


selected_rows = rows[start_index:end_index+1]

In [ ]:
print(selected_rows)

[['186', 'Марки. Picanto', '', 'Какие варианты комплектации доступны для модели Kia Picanto?', '', '', '', '', ' ', ' ', '', '', '', '', '', '', '', '', '', '', '', ''], ['187', 'Марки. Picanto', '', 'Какие опции входят в комплектацию Comfort модели Kia Picanto?', '', '', '', '', ' ', ' ', '', '', '', '', '', '', '', '', '', '', '', ''], ['188', 'Марки. Picanto', '', 'Какие особенности безопасности имеет Kia Picanto?', '', '', '', '', ' ', ' ', '', '', '', '', '', '', '', '', '', '', '', ''], ['189', 'Марки. Picanto', '', 'Какие двигатели доступны для Kia Picanto и какая их мощность?', '', '', '', '', ' ', ' ', '', '', '', '', '', '', '', '', '', '', '', ''], ['190', 'Марки. Picanto', '', 'Какой объем багажника у Kia Picanto и как он может быть увеличен?', '', '', '', '', ' ', ' ', '', '', '', '', '', '', '', '', '', '', '', ''], ['191', 'Марки. Picanto', '', 'Какие особенности комфорта предлагает Kia Picanto?', '', '', '', '', ' ', ' ', '', '', '', '', '', '', '', '', '', '', '', ''],

In [ ]:
temperature=0.3
verbose=0


answer = ''
for row in selected_rows:                #если все подряд надо for i, row in  enumerate(rows[1:], start=2):
  question_history = []
  dialog = ""

  print(row[0], row[3])
  rowno =int(row[0])

  user_question = row[3]
  answer = answer_index(user_question)


  sheet.update_cell(rowno+1, 5, answer)


print("Завершено!")

186 Какие варианты комплектации доступны для модели Kia Picanto?
187 Какие опции входят в комплектацию Comfort модели Kia Picanto?
188 Какие особенности безопасности имеет Kia Picanto?
189 Какие двигатели доступны для Kia Picanto и какая их мощность?
190 Какой объем багажника у Kia Picanto и как он может быть увеличен?
191 Какие особенности комфорта предлагает Kia Picanto?
192 Какие технологические возможности есть у Kia Picanto?
193 Какие функции поддержки связи с мобильными устройствами есть у Kia Picanto?
194 Где можно найти дополнительную информацию о комплектациях и опциях для Kia Picanto?
195 Какие скидки или специальные предложения доступны для покупки Kia Picanto?
196 Как работает ассистент движения в полосе
197 Что такое LFA
198 Какая скорость поддерживается системой ассистента движения в полосе (LFA) и как она использует камеру и радары-датчики для обеспечения безопасной дистанции и центрирования автомобиля на полосе?
199 В каких моделях есть ассистент движения
200 Как работа